<img src="figures/tweepy.png">

### **Table of Content:**

 1. [Installation](#head-1)  
 2. [Configuration](#head-2)   
 3. [Search Twitter for Tweets](#head-3)   
  3.1. [Tweets List to Pandas DataFrame](#head-3-1)   
 4. [Get tweets from a specific user](#head-4)
 
In this notebook, I'll explore the most popular twitter API wrapper library - Tweepy. I'll start from the scratch to set up the e scrapping pipeline, then save the ready to use results to a csv file for furthering modeling

To get started, you’ll need to do the following things:

- Set up a Twitter account
- Apply for Developer Access (https://developer.twitter.com) and then create an application that will generate the API credentials that you will use to access Twitter from Python.


Tutorial for applying for twitter API: https://cran.r-project.org/web/packages/rtweet/vignettes/auth.html

# 1. Installation <a class="anchor" id="head-1"></a>

Required libraries:

- pandas
- tweepy

In [ ]:
!pip install -r requirements.txt

# 2. Configuration <a class="anchor" id="head-2"></a>

Create a .env file that contains:

consumer_key= 'yourkeyhere'  <br>
consumer_secret= 'yourkeyhere' <br>
access_token= 'yourkeyhere' <br>
access_token_secret= 'yourkeyhere'

In [6]:
import os
import tweepy as tw
import pandas as pd

In [7]:
from dotenv import dotenv_values

# load environment variabele
config = dotenv_values(".env")

In [8]:
# configurate tweepy
auth = tw.OAuthHandler(config["consumer_key"], config["consumer_secret"])
auth.set_access_token(config["access_token"], config["access_token_secret"])
api = tw.API(auth, wait_on_rate_limit=True)

# 3. Search Twitter for Tweets <a class="anchor" id="head-3"></a>

In [9]:
# Define the search term and the date_since date as variables
search_words = "#wildfires"
date_since = "2018-11-16"

# Collect tweets
wildfires_tweets = tw.Cursor(api.search,
                       q=search_words,
                       lang="en",
                       since=date_since).items(5)

## 3. 1 Tweets List to Pandas DataFrame <a class="anchor" id="head-3-1"></a>

In [11]:
# fuction to extract data from tweet object
def extract_tweet_attributes(tweet_object):
    # create empty list
    tweet_list =[]
    # loop through tweet objects
    for tweet in tweet_object:
        tweet_id = tweet.id # unique integer identifier for tweet
        text = tweet.text # utf-8 text of tweet
        favorite_count = tweet.favorite_count
        retweet_count = tweet.retweet_count
        created_at = tweet.created_at # utc time tweet created
        source = tweet.source # utility used to post tweet
        reply_to_status = tweet.in_reply_to_status_id # if reply int of orginal tweet id
        reply_to_user = tweet.in_reply_to_screen_name # if reply original tweetes screenname
        retweets = tweet.retweet_count # number of times this tweet retweeted
        favorites = tweet.favorite_count # number of time this tweet liked
        # append attributes to list
        tweet_list.append({'tweet_id':tweet_id, 
                          'text':text, 
                          'favorite_count':favorite_count,
                          'retweet_count':retweet_count,
                          'created_at':created_at, 
                          'source':source, 
                          'reply_to_status':reply_to_status, 
                          'reply_to_user':reply_to_user,
                          'retweets':retweets,
                          'favorites':favorites})
    # create dataframe   
    df = pd.DataFrame(tweet_list, columns=['tweet_id',
                                           'text',
                                           'favorite_count',
                                           'retweet_count',
                                           'created_at',
                                           'source',
                                           'reply_to_status',
                                           'reply_to_user',
                                           'retweets',
                                           'favorites'])
    return df


wildfire_df = extract_tweet_attributes(wildfires_tweets)
wildfire_df.head()

,tweet_id,text,favorite_count,retweet_count,created_at,source,reply_to_status,reply_to_user,retweets,favorites
0,1388268182880886785,"As #California gears up for #FireSeason, check...",0,0,2021-04-30 23:05:02,Hootsuite Inc.,None,None,0,0
1,1388267172242337794,"RT @HaidaPrincess: 😯""@ABCWorldNews: 🔥#WILDFIRE...",0,2,2021-04-30 23:01:01,Twitter for iPhone,None,None,2,0
2,1388265466037096450,RT @Cal_OES: 🚨MUTUAL AID ALERT🚨 From the state...,0,13,2021-04-30 22:54:14,Twitter Web App,None,None,13,0
3,1388263950362501126,RT @DawnRoseTurner: @Mikethemikeman1 @earthma2...,0,2,2021-04-30 22:48:13,Twitter for iPhone,None,None,2,0
4,1388262947525472260,RT @EvanKirstel: #California can’t catch a bre...,0,16,2021-04-30 22:44:14,Twitter for iPhone,None,None,16,0


# 4. Get tweets from a specific user <a class="anchor" id="head-4"></a>

In [12]:
# tweets from a specific user
abc_tweets = api.user_timeline('ABC')

In [13]:
abc_df = extract_tweet_attributes(abc_tweets)
abc_df.head()

,tweet_id,text,favorite_count,retweet_count,created_at,source,reply_to_status,reply_to_user,retweets,favorites
0,1388268156737794048,The harassment was not by the soldier who alle...,7,6,2021-04-30 23:04:55,SocialFlow,NaN,None,6,7
1,1388262600299921409,The Biden administration is returning to the P...,380,83,2021-04-30 22:42:51,SocialFlow,NaN,None,83,380
2,1388258379232120835,Several thousand music fans have become the fi...,84,17,2021-04-30 22:26:04,SocialFlow,NaN,None,17,84
3,1388254117009764355,"As experts predicted, the pandemic is operatin...",40,14,2021-04-30 22:09:08,SocialFlow,NaN,None,14,40
4,1388249820800950286,Drug maker Pfizer says it will start sending U...,458,82,2021-04-30 21:52:04,SocialFlow,NaN,None,82,458
